In [20]:
# import the necessary packages
#from pyimagesearch.nms import non_max_suppression
#from pyimagesearch import config
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import pytesseract
import pandas as pd


In [2]:
MAX_PROPOSALS = 2000
MAX_PROPOSALS_INFER = 200
INPUT_DIMS = (224, 224)
MIN_PROBA = 0.99

In [4]:
def non_max_suppression(boxes, probabilities, overlap_thresh):
    if len(boxes) == 0:
        return []

    # Initialize the list of picked indexes
    pick = []

    # Grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Compute the area of the bounding boxes and sort the bounding
    # boxes by their probabilities
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(probabilities)

    # Keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # Grab the last index in the indexes list and add the index
        # value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Find the largest (x, y) coordinates for the start of the
        # bounding box and the smallest (x, y) coordinates for the
        # end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # Compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # Compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # Delete all indexes from the index list that have overlap
        # greater than the specified threshold
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    # Return only the bounding boxes and probabilities that were picked
    return boxes[pick], probabilities[pick]

In [14]:
MODEL_PATH= r"C:\Users\User\Documents\Table_Detection\table_detector_RCNN_MobileNet.h5"
ENCODER_PATH= r"C:\Users\User\Documents\Table_Detection\label_encoder_RCNN_MobileNet.pickle"

In [49]:
# load the our fine-tuned model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(MODEL_PATH)
lb = pickle.loads(open(ENCODER_PATH, "rb").read())
# load the input image from disk
image = cv2.imread(r"E:\TableBank2\1-s2.0-S0168851017300386-mmc1_0.jpg")
image = imutils.resize(image, width=500)
# run selective search on the image to generate bounding box proposal
# regions
print("[INFO] running selective search...")
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
rects = ss.process()
print("[INFO] Number of proposals:", len(rects))

# initialize the list of region proposals that we'll be classifying
# along with their associated bounding boxes
proposals = []
boxes = []
# loop over the region proposal bounding box coordinates generated by
# running selective search
for (x, y, w, h) in rects[:MAX_PROPOSALS_INFER]:
    # extract the region from the input image, convert it from BGR to
    # RGB channel ordering, and then resize it to the required input
    # dimensions of our trained CNN
    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, INPUT_DIMS,
                     interpolation=cv2.INTER_CUBIC)
    # further preprocess the ROI
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    # update our proposals and bounding boxes lists
    proposals.append(roi)
    boxes.append((x, y, x + w, y + h))
# convert the proposals and bounding boxes into NumPy arrays
proposals = np.array(proposals, dtype="float32")
print("[INFO] proposals shape:", proposals.shape)

boxes = np.array(boxes, dtype="int32")
print("[INFO] proposal shape: {}".format(proposals.shape))
# classify each of the proposal ROIs using fine-tuned model
print("[INFO] classifying proposals...")
proba = model.predict(proposals)
print("[INFO] Available classes:", lb.classes_)

print("[INFO] applying NMS...")
labels = lb.classes_[np.argmax(proba, axis=1)]
idxs = np.where(labels == "table")[0]
# use the indexes to extract all bounding boxes and associated class
# label probabilities associated with the "raccoon" class
boxes = boxes[idxs]
proba = proba[idxs][:, 1]
# further filter indexes by enforcing a minimum prediction
# probability be met
idxs = np.where(proba >= MIN_PROBA)
boxes = boxes[idxs]
proba = proba[idxs]

# clone the original image so that we can draw on it
clone_after_nms = image.copy()
clone_before_nms = image.copy()
# loop over the bounding boxes and associated probabilities
for (box, prob) in zip(boxes, proba):
    # draw the bounding box, label, and probability on the image
    (startX, startY, endX, endY) = box
    cv2.rectangle(clone_before_nms, (startX, startY), (endX, endY),
                  (0, 255, 0), 2)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    text = "table: {:.2f}%".format(prob * 100)
    cv2.putText(clone_before_nms, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
# show the output after *before* running NMS
cv2.imshow("Before NMS", clone_before_nms)

# Apply non-maximum suppression
overlap_thresh = 0.1  # Adjust this threshold as needed
boxes_nms, proba_nms = non_max_suppression(boxes, proba, overlap_thresh)

# Draw the remaining bounding boxes after NMS
for (box, prob) in zip(boxes_nms, proba_nms):
    # Draw the bounding box, label, and probability on the image
    (startX, startY, endX, endY) = box
    cv2.rectangle(clone_after_nms, (startX, startY), (endX, endY), (0, 255, 0), 2)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    text = "table: {:.2f}%".format(prob * 100)
    cv2.putText(clone_after_nms, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

cv2.imshow("After NMS", clone_after_nms)


# Wait for a key press and close all OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] loading model and label binarizer...
[INFO] running selective search...
[INFO] Number of proposals: 1072
[INFO] proposals shape: (200, 224, 224, 3)
[INFO] proposal shape: (200, 224, 224, 3)
[INFO] classifying proposals...
7/7 [==============================] - 4s 502ms/step
[INFO] Available classes: ['no_table' 'table']
[INFO] applying NMS...


In [17]:
!pip install pytesseract

  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)


In [50]:


# Load your fine-tuned model and label binarizer
print("[INFO] loading model and label binarizer...")
model = load_model(MODEL_PATH)
lb = pickle.loads(open(ENCODER_PATH, "rb").read())

# Load the input image
image = cv2.imread(r"E:\TableBank2\1-s2.0-S0168851017300386-mmc1_0.jpg")
image = imutils.resize(image, width=500)

# Run selective search to generate bounding box proposal regions
print("[INFO] running selective search...")
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
rects = ss.process()
print("[INFO] Number of proposals:", len(rects))

# Initialize the list of region proposals that we'll be classifying
# along with their associated bounding boxes
proposals = []
boxes = []

# Loop over the region proposal bounding box coordinates generated by
# running selective search
for (x, y, w, h) in rects[:MAX_PROPOSALS_INFER]:
    # Extract the region from the input image, convert it from BGR to
    # RGB channel ordering, and then resize it to the required input
    # dimensions of our trained CNN
    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, INPUT_DIMS, interpolation=cv2.INTER_CUBIC)
    # Further preprocess the ROI
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    # Update our proposals and bounding boxes lists
    proposals.append(roi)
    boxes.append((x, y, x + w, y + h))

# Convert the proposals and bounding boxes into NumPy arrays
proposals = np.array(proposals, dtype="float32")
print("[INFO] proposals shape:", proposals.shape)

boxes = np.array(boxes, dtype="int32")
print("[INFO] proposal shape: {}".format(proposals.shape))

# Classify each of the proposal ROIs using the fine-tuned model
print("[INFO] classifying proposals...")
proba = model.predict(proposals)
print("[INFO] Available classes:", lb.classes_)

# Find the index of all predictions that are positive for the "table" class
print("[INFO] applying NMS...")
labels = lb.classes_[np.argmax(proba, axis=1)]
idxs = np.where(labels == "table")[0]

# Use the indexes to extract all bounding boxes and associated class
# label probabilities associated with the "table" class
boxes = boxes[idxs]
proba = proba[idxs][:, 1]

# Further filter indexes by enforcing a minimum prediction
# probability be met
idxs = np.where(proba >= MIN_PROBA)
boxes = boxes[idxs]
proba = proba[idxs]

# ... (previous code)

# Clone the original image so that we can draw on it
clone_after_nms = image.copy()

# Function to enhance image quality
def enhance_image(image):
    # Convert the image to grayscale (assuming it's color)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    
    return blurred

# Apply non-maximum suppression
overlap_thresh = 0.1  # Adjust this threshold as needed
boxes_nms, proba_nms = non_max_suppression(boxes, proba, overlap_thresh)

# Initialize a list to store the extracted table images
table_images = []

# Loop over the bounding boxes and associated probabilities after NMS
for (box, prob) in zip(boxes_nms, proba_nms):
    # Extract the region corresponding to the table
    (startX, startY, endX, endY) = box
    table_roi = image[startY:endY, startX:endX]

    # Enhance the quality of the table_roi
    enhanced_table_roi = enhance_image(table_roi)

    # Perform OCR on the enhanced table_roi to extract the table data
    extracted_data = pytesseract.image_to_string(enhanced_table_roi)

    # Append the extracted data to the list (or save the image if needed)
    table_images.append(enhanced_table_roi)

# Optionally, you can display or save the extracted and enhanced table images
for i, table_image in enumerate(table_images):
    cv2.imshow(f"Table {i + 1}", table_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Optionally, you can also save the extracted and enhanced table images to files
for i, table_image in enumerate(table_images):
    cv2.imwrite(f"table_{i + 1}.jpg", table_image)

# Optionally, you can perform further text processing or analysis on the extracted data
for i, extracted_data in enumerate(table_data):
    print(f"Table {i + 1} Extracted Data:")
    print(extracted_data)
    print("\n")

[INFO] loading model and label binarizer...
[INFO] running selective search...
[INFO] Number of proposals: 1072
[INFO] proposals shape: (200, 224, 224, 3)
[INFO] proposal shape: (200, 224, 224, 3)
[INFO] classifying proposals...
7/7 [==============================] - 4s 518ms/step
[INFO] Available classes: ['no_table' 'table']
[INFO] applying NMS...
Table 1 Extracted Data:
['ond int website']


Table 2 Extracted Data:
['oer ae']


Table 3 Extracted Data:
['“Supplementary Table | Onkns forum with publications on the clalieod obesity araegy within 24 hows of', 'bin af he ty', 'aT RT', 'aaa']


Table 4 Extracted Data:
[]


Table 5 Extracted Data:
[]


Table 6 Extracted Data:
[]


Table 7 Extracted Data:
[]


Table 8 Extracted Data:
['“Supplementary Table | Online Toran wit pabcations on De allied oben aeay within 24 boas ot', 'bin af he ty', 'aT RT', 'aaa']


Table 9 Extracted Data:
['| pene ae Lene an ne', 'bln of he egy', 'ond int website', 'aaa']


Table 10 Extracted Data:
['Tie Pc ect

In [54]:
# load the our fine-tuned model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(MODEL_PATH)
lb = pickle.loads(open(ENCODER_PATH, "rb").read())
# load the input image from disk
image = cv2.imread(r"E:\TableBank\Detection\images\%5BMS-GPOL%5D-170601_73.jpg")
image = imutils.resize(image, width=500)
# run selective search on the image to generate bounding box proposal
# regions
print("[INFO] running selective search...")
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
rects = ss.process()
print("[INFO] Number of proposals:", len(rects))

# initialize the list of region proposals that we'll be classifying
# along with their associated bounding boxes
proposals = []
boxes = []
# loop over the region proposal bounding box coordinates generated by
# running selective search
for (x, y, w, h) in rects[:MAX_PROPOSALS_INFER]:
    # extract the region from the input image, convert it from BGR to
    # RGB channel ordering, and then resize it to the required input
    # dimensions of our trained CNN
    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, INPUT_DIMS,
                     interpolation=cv2.INTER_CUBIC)
    # further preprocess the ROI
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    # update our proposals and bounding boxes lists
    proposals.append(roi)
    boxes.append((x, y, x + w, y + h))
# convert the proposals and bounding boxes into NumPy arrays
proposals = np.array(proposals, dtype="float32")
print("[INFO] proposals shape:", proposals.shape)

boxes = np.array(boxes, dtype="int32")
print("[INFO] proposal shape: {}".format(proposals.shape))
# classify each of the proposal ROIs using fine-tuned model
print("[INFO] classifying proposals...")
proba = model.predict(proposals)
print("[INFO] Available classes:", lb.classes_)
# find the index of all predictions that are positive for the
# "raccoon" class
print("[INFO] applying NMS...")
labels = lb.classes_[np.argmax(proba, axis=1)]
idxs = np.where(labels == "table")[0]
# use the indexes to extract all bounding boxes and associated class
# label probabilities associated with the "raccoon" class
boxes = boxes[idxs]
proba = proba[idxs][:, 1]
# further filter indexes by enforcing a minimum prediction
# probability be met
idxs = np.where(proba >= MIN_PROBA)
boxes = boxes[idxs]
proba = proba[idxs]

# clone the original image so that we can draw on it
clone_after_nms = image.copy()
clone_before_nms = image.copy()
# loop over the bounding boxes and associated probabilities
for (box, prob) in zip(boxes, proba):
    # draw the bounding box, label, and probability on the image
    (startX, startY, endX, endY) = box
    cv2.rectangle(clone_before_nms, (startX, startY), (endX, endY),
                  (0, 255, 0), 2)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    text = "table: {:.2f}%".format(prob * 100)
    cv2.putText(clone_before_nms, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
# show the output after *before* running NMS
cv2.imshow("Before NMS", clone_before_nms)

# Apply non-maximum suppression
overlap_thresh = 0.1  # Adjust this threshold as needed
boxes_nms, proba_nms = non_max_suppression(boxes, proba, overlap_thresh)

# Draw the remaining bounding boxes after NMS
for (box, prob) in zip(boxes_nms, proba_nms):
    # Draw the bounding box, label, and probability on the image
    (startX, startY, endX, endY) = box
    cv2.rectangle(clone_after_nms, (startX, startY), (endX, endY), (0, 255, 0), 2)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    text = "table: {:.2f}%".format(prob * 100)
    cv2.putText(clone_after_nms, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

cv2.imshow("After NMS", clone_after_nms)


# Wait for a key press and close all OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] loading model and label binarizer...
[INFO] running selective search...
[INFO] Number of proposals: 1614
[INFO] proposals shape: (200, 224, 224, 3)
[INFO] proposal shape: (200, 224, 224, 3)
[INFO] classifying proposals...
7/7 [==============================] - 4s 473ms/step
[INFO] Available classes: ['no_table' 'table']
[INFO] applying NMS...
